In [ ]:
%matplotlib inline
from arc_maps import get_data
from smif.data_layer import Results
import seaborn as sbn
import matplotlib.pyplot as plt
import numpy as np
results = Results({'interface': 'local_csv', 'dir': '../'})

sbn.set_context('talk', font_scale=0.8, rc={"lines.linewidth": 2.5})
sbn.set_palette('Paired')

In [ ]:
filepath = '../credential.ini'
arc_lads = get_data.get_lads(filepath)

# Total transmission-connected generation

In [ ]:
elec_outputs = [x for x in results.list_outputs('energy_supply_constrained') if x.startswith('tran_') and not '_curtailed' in x]
elec_outputs

In [ ]:
model_run_names = ['arc_ets__baseline', 'arc_ets__new-cities', 'arc_ets__expansion']
model_names = ['energy_supply_constrained']
timesteps=[2015, 2030, 2050]

df = results.read_results(
        model_run_names, model_names, elec_outputs, timesteps).drop(
    columns=['decision'])
df = df.melt(id_vars=['model_run', 'timestep', 'bus_bars', 'seasonal_week'])
df['model_run'] = df['model_run'].str.replace('arc_ets__', '')
data = df.groupby(
    by=['model_run', 'timestep', 'seasonal_week', 'variable']).sum().drop(
    columns='bus_bars').reset_index()
data.head()

In [ ]:
filtered = data[data['seasonal_week'].isin(range(0,25))]
fig = sbn.relplot(data=filtered, x='seasonal_week', y='value', col='model_run', row='timestep', 
            kind='line', hue='variable')
fig.savefig('generation_mix.png', transparent=True)

In [ ]:
mapping = {'tran_coal': 'fossil',
           'tran_gas_fired': 'fossil',
           'tran_hydro': 'renewable',
           'tran_interconnector': 'fossil',
           'tran_nuclear': 'nuclear',
           'tran_pump_power': 'renewable',
           'tran_pv_power': 'renewable',
           'tran_renewable': 'renewable',
           'tran_wind_offshore': 'renewable',
           'tran_wind_onshore': 'renewable'}

filtered = data[data['seasonal_week'].isin(range(0, 169))]
filtered['type'] = filtered['variable'].apply(lambda x: mapping[x])
filtered = filtered.groupby(by=['model_run', 'timestep', 'seasonal_week', 'type']).sum()
fig = sbn.relplot(data=filtered.reset_index(), x='seasonal_week', y='value', row='model_run', col='timestep', 
            kind='line', hue='type')
fig.savefig('es_summary_gen_mix.png', transparent=True)

# Results: Total emissions

In [ ]:
def get_total_results(model_run_names, model_names, output_name, timesteps):
    es_res = results.read_results(
            model_run_names, model_names, [output_name], timesteps)
    agg_results = es_res.drop(
        columns='decision').groupby(
        by=['model_run', 'timestep']).sum().reset_index()
    return agg_results[['model_run', 'timestep', output_name]].set_index(['model_run', 'timestep'])

model_names = ['energy_supply_constrained']
output_name = 'e_emissions_eh'
timesteps=[2015, 2030, 2050]
    
e_emissions_eh = get_total_results(model_run_names, model_names, 'e_emissions_eh', timesteps)
e_emissions_bb = get_total_results(model_run_names, model_names, 'e_emissions', timesteps)
h_emissions_eh = get_total_results(model_run_names, model_names, 'h_emissions_eh', timesteps)

# fig = sbn.relplot(data=emissions_eh, row='model_run', x='timestep', y=output_name, kind='line')


In [ ]:
emissions = e_emissions_bb.join(e_emissions_eh).join(h_emissions_eh)
emissions['total'] = emissions['e_emissions'] + emissions['e_emissions_eh'] + emissions['h_emissions_eh']
data = emissions.reset_index().melt(id_vars=['model_run', 'timestep'])
data['model_run'] = data['model_run'].str.replace('arc_ets__', '')
fig = sbn.relplot(data=data[data['variable'] == 'total'], x='timestep', hue='model_run', y='value', kind='line')
fig.set_axis_labels('year', 'emissions (tCO2)')
fig.savefig('total_emissions.png', transparent=True)

# Results: operational cost

In [ ]:
output_name = 'total_opt_cost'
    
cost = get_total_results(model_run_names, model_names, output_name, timesteps)
fig = sbn.catplot(data=cost.reset_index(), x='timestep', col='model_run', y=output_name, kind='bar')
fig.set_axis_labels('year', 'operational cost (£M)')

# Total distributed generation

In [ ]:
elec_outputs = [x for x in results.list_outputs('energy_supply_constrained') if x.startswith('eh_') ]
elec_outputs